# Scaling Dashboards

<img src=https://assets.website-files.com/5c197923e5851742d9bc835d/5cab8777229f4d375124ad2b_QIJCHoKCigaFRrp_mA2bmSWeZ7wqTHTIyy5zmkZyOgam3c6ZS0X-wCkh4hV236E2T8WCKulFC2NUWH-6xKzQlsTugZuNHEhBuXk7kTGaBPOCtGepD6fxggV2X35wSuIM2oMcJnne.png />

Once your dashboard is in front of it’s full audience how the dashboard is used is likely to evolve. This can be an expansion of decisions they would like to see supported or the number of groups who wants to use it for their specific scenarios. To accommodate these changes there are some scaling strategies to consider using.

## Linking out

If the feedback for the dashboard is to support more decisions consider if it is appropriate for the dashboard you current have or if you should start the dashboard design process over to create a second dashboard to support these new decisions. You can then provide links on the original dashboard to link to these new dashboards.

<img src=https://assets.website-files.com/5c197923e5851742d9bc835d/5cab877723c44c0baec0b35f_sjfLwTQaK3WCk7K_ZHIUSJOEOM3N1nsQuzhst26zBpv7MHyvagA2BL5iHvziHGVXEPZNrWIDekB8grpzOsZ-SLKCwfB-ArRK1kvQu-g9_iaeMCrVldi0bpzkMFxyNP6x2vnnGIhG.png />

There are also technical benefits to linking out. Keeping the number of queries limited per dashboard will keep the dashboard loading quickly.

## Interactivity

If the feedback for the dashboard is to support more groups’ specific scenarios you will need to incorporate interactive features. This means having dropdowns for variables so that multiple situations can be evaluated using the same framework of the dashboard.

<img src=https://assets.website-files.com/5c197923e5851742d9bc835d/5cab8777229f4d6c1024ad4d_IiyDfXn_1cyrrj73cIq44Q2C1MCldA9OBPbAL01zq3JJhRcNWECd4xoptUZn2U38IzDJfO5lmy9bBGtqHUKi-sLM7hqxf0dh8OTOELJLhw4eJ0_jec4-CVpu-RWwEX-eg-NdAHmx.png />

Here we have an example where at the top there is an interactive element to change the date range we are viewing.

When you introduce interactivity it is a best practice to turn off any auto refresh if you have multiple variables you can set. This will limit the amount of queries being performed until you confirm to refresh the dashboard.


## Optimization

Regardless of how it evolves if the usage on a dashboard is high the demand on your database will likely increase. You need to ensure the dashboard still loads quickly and that the work placed on the database is mitigated. This can be accomplished by:

- Optimizing queries
- Setting the schedule
- Removing unused queries

### Optimizing queries

At Chartio our rule of thumb is that if a query takes longer than 30 seconds there likely can be something done to optimize your query. If aggregations are taking a long time go to the Data Gatekeeper to discuss creating a pre-aggregated table that you can query from. This sort of data modeling can drastically improve query performance.

In addition leave any data manipulation (truncation, casting, etc) until after the aggregation. This means that you will aggregate the data first and then apply the transformations to the aggregated data.

```SQL
SELECT SUM(num), category
FROM table
GROUP BY CAST(category AS VARCHAR)
```

```SQL
SELECT SUM(num), CAST(category AS VARCHAR)
FROM table
GROUP BY category
```

The bottom query is much more efficient because it groups by category before applying sum and cast transformations.

### Order of a SQL query

The way to make a query run faster is to reduce the number of calculations that the software must perform. To do this, first you’ll need some understanding of how SQL actually executes a query. Let’s take a look at a sample SQL query:

```SQL
SELECT DISTINCT column, AGGREGATE(column)
FROM table1
JOIN table2
ON table1.column = table2.column
WHERE constraint_expression
GROUP BY column
HAVING constraint_expression
ORDER BY column ASC/DESC
LIMIT count;
```

Each part of the query is executed sequentially, so it’s important to understand the order of execution:

1. **FROM and JOIN:** The FROM clause, and subsequent JOINs are first executed to determine the total working set of data that is being queried
2. **WHERE:** Once we have the total working set of data, the WHERE constraints are applied to the individual rows, and rows that do not satisfy the constraint are discarded.
3. **GROUP BY:** The remaining rows after the WHERE constraints are applied are then grouped based on common values in the column specified in the GROUP BY clause.
4. **HAVING:** If the query has a GROUP BY clause, then the constraints in the HAVING clause are applied to the grouped rows, and the grouped rows that don’t satisfy the constraint are discarded.
5. **SELECT:** Any expressions in the SELECT part of the query are finally computed.
6. **DISTINCT:** Of the remaining rows, rows with duplicate values in the column marked as DISTINCT will be discarded.
7. **ORDER BY:** If an order is specified by the ORDER BY clause, the rows are then sorted by the specified data in either ascending or descending order.
8. **LIMIT:** Finally, the rows that fall outside the range specified by the LIMIT are discarded, leaving the final set of rows to be returned from the query.

Now that we understand the basic structure and order of a SQL query, we can take a look at the tips to optimize them for faster processing.

### 8 tips for faster querying

1. **Define SELECT fields instead of SELECT \*:** If a table has many fields and rows, selecting all the columns (by using SELECT \*) over-utilizes the database resources in querying a lot of unnecessary data. Defining fields in the SELECT statement will point the database to querying only the required data to solve the business problem.
2. **Avoid SELECT DISTINCT if possible:** SELECT DISTINCT works by grouping all fields in the query to create distinct results. To accomplish this goal however, a large amount of processing power is required.
3. **Use WHERE instead of HAVING to define Filters:** As per the SQL order of operations, HAVING statements are calculated after WHERE statements. If we need to filter a query based on conditions, a WHERE statement is more efficient.
4. **Use WILDCARDS at the end of the phrase:** When a leading wildcard is used, especially in combination with an ending wildcard, the database is tasked with searching all records for a match anywhere within the selected field. Consider this query to pull cities beginning with ‘Char’:
    ```SQL
    SELECT City FROM Customers
    WHERE City LIKE ‘%Char%’
    ```
    
    A more efficient query would be:
    
    ```SQL
    SELECT City FROM Customers
    WHERE City LIKE ‘Char%’
    ```
5. **Use LIMIT to sample query results:** Before running a query for the first time, ensure the results will be desirable and meaningful by using a LIMIT statement.
6. **Run Queries During Off-Peak Times:** Heavier queries which take a lot of database load should run when concurrent users are at their lowest number, which is typically during the middle of the night.
7. **Replace SUBQUERIES with JOIN:** Although subqueries are useful, they often can be replaced by a join, which is definitely faster to execute. Consider the example below:
    ```SQL
    SELECT a.id,
    (SELECT MAX(created)
    FROM posts
    WHERE author_id = a.id)
    AS latest_post FROM authors a
    ```    
    
    To avoid the sub-query, it can be rewritten with a join as:
    
    ```SQL
    SELECT a.id, MAX(p.created) AS latest_post
    FROM authors a
    INNER JOIN posts p
    ON (a.id = p.author_id)
    GROUP BY a.id
    ```
    
8. **Index your tables properly:** Proper indexing can make a slow database perform better. Conversely, improper indexing can make a high-performing database run poorly. The difference depends on how you structure the indexes. You should create an index on a column in any of the following situations:
    - The column is queried frequently
    - Foreign key column(s) that reference other tables
    - A unique key exists on the column(s)

When querying a production database, optimization is key. An inefficient query may pose a burden on the production database’s resources, and cause slow performance or loss of service for other users if the query contains errors.

When optimizing your database server, you need to tune the performance of individual queries. This is even more important than tuning other aspects of your server installation that affect performance, such as hardware and software configurations. Even if your database server runs on the most powerful hardware available, its performance can be negatively affected by a handful of misbehaving queries.


### Setting the schedule
Most business dashboards are not using real time data. Data is delivered from the data warehouse where you are querying from on a schedule in batches, which is setup by the Data Gatekeeper. Set expectations with the audience about how “live” the data is. You can also discuss with the Data Gatekeeper the speed at which people need to make decisions based on the data so they can adjust the schedule for when data is loaded into the data warehouse.

### Removing unused dashboards
While this dashboard may be getting heavily used, it is likely some are not being used as much. It is a best practice to regularly archive dashboards that have not been viewed in over 90 days. This is because the more dashboards are querying the database, the more the database slows  down for everyone. We do recommend sending out an email first alerting people which dashboards will be archived so they can respond and flag any that shouldn’t be.



## Documentation
Beyond the audience there is another consumer of your dashboard. This is future analysts who are trying to build their own dashboards. This could also be a future you. Do the future a favor and document your queries so that they can be understood easily and any quirks can be identified quickly.

A few settings will make your impact on the future much higher. Make sure that other people have access to the query and that they can access the data sources used. If these are protected sources, include the information about how to get access from the Data Gatekeeper

## Summary
- Link out to support more decisions
- Add interactivity to accommodate more individuals/groups needing to make the same decisions
- Set a refresh rate that mimics the rate of decision making so that it is not refreshing more than necessary
- Document your work so that you and others can learn from your query decisions in the future.